# Reducción del dataset estratégica y Join con datos del suelo de cada sitio de interés

In [1]:
import numpy as np
import pandas as pd

In [2]:
np.random.seed(666)

In [3]:
df_train = pd.read_csv('data/train_timeseries_interpolated.csv')
df_val = pd.read_csv('data/val_timeseries_interpolated.csv')
df_test = pd.read_csv('data/test_timeseries_interpolated.csv')

In [4]:
# Cargamos soil data
df_soil = pd.read_csv('data/soil_data.csv')

In [5]:
df_soil.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3109 entries, 0 to 3108
Data columns (total 32 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   fips           3109 non-null   int64  
 1   lat            3109 non-null   float64
 2   lon            3109 non-null   float64
 3   elevation      3109 non-null   int64  
 4   slope1         3109 non-null   float64
 5   slope2         3109 non-null   float64
 6   slope3         3109 non-null   float64
 7   slope4         3109 non-null   float64
 8   slope5         3109 non-null   float64
 9   slope6         3109 non-null   float64
 10  slope7         3109 non-null   float64
 11  slope8         3109 non-null   float64
 12  aspectN        3109 non-null   float64
 13  aspectE        3109 non-null   float64
 14  aspectS        3109 non-null   float64
 15  aspectW        3109 non-null   float64
 16  aspectUnknown  3109 non-null   float64
 17  WAT_LAND       3109 non-null   float64
 18  NVG_LAND

In [6]:
pd.set_option('display.max_rows', None)

In [7]:
# Eliminar las columnas 'score' y 'score_interpolated'
columns_to_drop = ['score', 'score_interpolated']
df_train = df_train.drop(columns=columns_to_drop)
df_val = df_val.drop(columns=columns_to_drop)
df_test = df_test.drop(columns=columns_to_drop)

In [8]:
df_train['date'] = pd.to_datetime(df_train['date'])
df_val['date'] = pd.to_datetime(df_val['date'])
df_test['date'] = pd.to_datetime(df_test['date'])

In [9]:
df_combined = pd.concat([df_train, df_val, df_test])

In [10]:
df_combined = df_combined.reset_index(drop=True)

In [11]:
# Nos quedamos aleatoriamente con 500 fips únicos
fips_to_keep = np.random.choice(df_combined['fips'].unique(), size=500, replace=False)

# Filtrar el DataFrame para que solo contenga los 500 fips seleccionados
df_combined_500_fips = df_combined[df_combined['fips'].isin(fips_to_keep)]

In [12]:
# Filtrar los fips restantes (aquellos que no están en fips_to_keep)
fips_to_exclude = df_combined['fips'].unique()[~np.isin(df_combined['fips'].unique(), fips_to_keep)]

# Guardar el DataFrame con los 2500 fips restantes
df_remaining_fips = df_combined[df_combined['fips'].isin(fips_to_exclude)]



In [13]:
df_combined.shape

(10219104, 21)

In [14]:
len(df_combined.fips.unique())

3108

In [15]:
df_combined_500_fips.shape

(1644000, 21)

In [16]:
len(df_combined_500_fips.fips.unique())

500

In [17]:
df_remaining_fips.shape

(8575104, 21)

In [18]:
len(df_remaining_fips.fips.unique())

2608

In [19]:
# Joinear a df_combined_500_fips el df_soil por fips (para los 500 fips seleccionados)
df_combined_500_fips = df_combined_500_fips.merge(df_soil, on='fips', how='left')

In [20]:
# Joinear a df_remaining_fips el df_soil por fips (para los 2500 fips restantes)
df_remaining_fips = df_remaining_fips.merge(df_soil, on='fips', how='left')

In [21]:
df_combined_500_fips.shape

(1644000, 52)

In [22]:
df_remaining_fips.shape

(8575104, 52)

In [18]:
df_combined_500_fips.to_parquet('data/combined_timeseries_interpolated.parquet', index=False)

In [23]:
df_remaining_fips.to_parquet('data/remaining_timeseries_interpolated_2500_fips.parquet', index=False)